## TensorFlow2教程-用keras构建自己的网络层
### 1 构建一个简单的网络层
我们可以通过继承tf.keras.layer.Layer,实现一个自定义的网络层。

In [1]:
from __future__ import absolute_import, print_function, division
import tensorflow as tf
tf.keras.backend.clear_session()
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

In [2]:
# 定义网络层就是：设置网络权重和输入到输出的计算过程
from turtle import shape


class MyLayer(layers.Layer):
    def __init__(self, input_dim=32, unit=32):
        super(MyLayer, self).__init__()
        w_init = tf.random_normal_initializer()
        # 权重变量
        self.weight = tf.Variable(
            initial_value=w_init(shape=(input_dim, unit), dtype=tf.float32),
            trainable=True
        )
        # 偏置变量
        b_init = tf.zeros_initializer()
        self.bias = tf.Variable(
            initial_value=b_init(shape=(unit,), dtype=tf.float32),
            trainable=True
        )

    def call(self, inputs):
        # 全连接网络
        return tf.matmul(inputs, self.weight) + self.bias

x = tf.ones((3,5))
my_layer = MyLayer(5, 4)
out = my_layer(x)
print(out)

tf.Tensor(
[[-0.12649278 -0.0621473   0.09325793 -0.1717659 ]
 [-0.12649278 -0.0621473   0.09325793 -0.1717659 ]
 [-0.12649278 -0.0621473   0.09325793 -0.1717659 ]], shape=(3, 4), dtype=float32)


按上面构建网络层，图层会自动跟踪权重w和b，当然我们也可以直接用add_weight的方法构建权重

In [3]:
class MyLayer(layers.Layer):
    def __init__(self, input_dim=32, unit=32):
        super(MyLayer, self).__init__()
        # 使用add_weight添加网络变量，使其可追踪
        self.weight = self.add_weight(
            shape=(input_dim, unit),
            initializer=keras.initializers.RandomNormal(),
            trainable=True
        )
        self.bias = self.add_weight(
            shape=(unit, ),
            initializer=keras.initializers.Zeros(),
            trainable=True
        )
    
    def call(self, inputs):
        return tf.matmul(inputs, self.weight) + self.bias
    
x = tf.ones((3,5))
my_layer = MyLayer(5, 4)
out = my_layer(x)
print(out)

tf.Tensor(
[[-0.05799209  0.03156646  0.08374223 -0.11691136]
 [-0.05799209  0.03156646  0.08374223 -0.11691136]
 [-0.05799209  0.03156646  0.08374223 -0.11691136]], shape=(3, 4), dtype=float32)


也可以设置不可训练的权重

In [4]:
class AddLayer(layers.Layer):
    def __init__(self, input_dim=32):
        super(AddLayer, self).__init__()
        # 只存储，不训练的变量
        self.sum = self.add_weight(shape=(input_dim,),
                                initializer=keras.initializers.Zeros(),
                                trainable=False)
       
    
    def call(self, inputs):
        self.sum.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.sum
        
x = tf.ones((3,3))
my_layer = AddLayer(3)
out = my_layer(x)
print(out.numpy())
out = my_layer(x)
print(out.numpy())
print('weight:', my_layer.weights)
print('non-trainable weight:', my_layer.non_trainable_weights)
print('trainable weight:', my_layer.trainable_weights)

[3. 3. 3.]
[6. 6. 6.]
weight: [<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([6., 6., 6.], dtype=float32)>]
non-trainable weight: [<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([6., 6., 6.], dtype=float32)>]
trainable weight: []


当定义网络时不知道网络的维度是可以重写build()函数，用获得的shape构建网络

In [5]:
class MyLayer(layers.Layer):
    def __init__(self, unit=32):
        super(MyLayer, self).__init__()
        self.unit = unit
    
    def build(self, input_shape):
        # 在build时获取input_shape
        self.weight = self.add_weight(shape=(input_shape[-1], self.unit),
                                     initializer=keras.initializers.RandomNormal(),
                                     trainable=True)
        self.bias = self.add_weight(shape=(self.unit,),
                                   initializer=keras.initializers.Zeros(),
                                   trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.weight) + self.bias
        

my_layer = MyLayer(3)
x = tf.ones((3,5))
out = my_layer(x)
print(out)

my_layer = MyLayer(3)
x = tf.ones((2,2))
out = my_layer(x)
print(out)

tf.Tensor(
[[ 0.00701024 -0.06287038 -0.30136997]
 [ 0.00701024 -0.06287038 -0.30136997]
 [ 0.00701024 -0.06287038 -0.30136997]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[ 0.04437653  0.06346857 -0.00314982]
 [ 0.04437653  0.06346857 -0.00314982]], shape=(2, 3), dtype=float32)


### 2 使用子层递归构建网络层
可以在自定义网络层中调用其他自定义网络层

In [6]:
class MyBlock(layers.Layer):
    def __init__(self):
        super(MyBlock, self).__init__()
        # 其他自定义网络层
        self.layer1 = MyLayer(32)
        self.layer2 = MyLayer(16)
        self.layer3 = MyLayer(2)
    
    def call(self, inputs):
        h1 = self.layer1(inputs)
        h1 = tf.nn.relu(h1)
        h2 = self.layer2(h1)
        h2 = tf.nn.relu(h2)
        return self.layer3(h2)
    
my_block = MyBlock()
print('trainable weights:', len(my_block.trainable_weights))
y = my_block(tf.ones(shape=(3, 64)))
# 构建网络在build()里面，所以执行了才有网络
print('trainable weights:', len(my_block.trainable_weights)) 

trainable weights: 0
trainable weights: 6


可以通过构建网络层的方法来收集loss，并可以递归调用。

In [8]:
class LossLayer(layers.Layer):
    def __init__(self, rate=1e-2):
        super(LossLayer, self).__init__()
        self.rate = rate
    
    def call(self, inputs):
        # 添加loss
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs

class OutLayer(layers.Layer):
    def __init__(self):
        super(OutLayer, self).__init__()
        self.loss_fun=LossLayer(1e-2)
    
    def call(self, inputs):
        # 就一个loss层
        return self.loss_fun(inputs)

my_layer = OutLayer()
print(len(my_layer.losses)) # 未执行call
y = my_layer(tf.zeros(1,1))
print(len(my_layer.losses)) # 执行call之后
y = my_layer(tf.zeros(1,1))
print(len(my_layer.losses)) # call之前会重置为0


0
1
1


如果中间调用了keras网络层，里面的正则化loss也会被加入进来

In [9]:
class OuterLayer(layers.Layer):

    def __init__(self):
        super(OuterLayer, self).__init__()
        # 子层中正则化loss也会添加到总的loss中
        self.dense = layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l2(1e-3))
    
    def call(self, inputs):
        return self.dense(inputs)

my_layer = OuterLayer()
y = my_layer(tf.zeros((1,1)))
print(my_layer.losses) 
print(my_layer.weights) 

[<tf.Tensor: shape=(), dtype=float32, numpy=0.001910928>]
[<tf.Variable 'outer_layer/dense/kernel:0' shape=(1, 32) dtype=float32, numpy=
array([[-0.07910636,  0.10402972,  0.27718008, -0.33846265, -0.00764611,
         0.10265589,  0.33764917, -0.17633584,  0.27937973,  0.11188376,
         0.3739105 ,  0.2025696 ,  0.24472874, -0.03522751,  0.27047437,
        -0.09395269, -0.39600804, -0.39767784, -0.0683735 , -0.0143874 ,
         0.11743826, -0.21127768,  0.03196621, -0.1056419 , -0.3340903 ,
         0.38742024, -0.10020471, -0.3868774 ,  0.1689626 , -0.11181715,
        -0.36308247,  0.384009  ]], dtype=float32)>, <tf.Variable 'outer_layer/dense/bias:0' shape=(32,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>]


## 3 其他网络层配置
### 3.1 使自己的网络层可以序列化

In [24]:
class Linear(layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.units = units
    
    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer=keras.initializers.RandomNormal(),
            trainable=True
        )
        self.b = self.add_weight(
            shape=(self.units, ),
            initializer=keras.initializers.RandomNormal(),
            trainable=True
        )
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    def get_config(self):
        # 获取网络配置实现序列化
        config = super(Linear, self).get_config()
        config.update({'units': self.units})
        return config

linear_layer = Linear(units=64)
config = linear_layer.get_config()
print(config)

{'name': 'linear_4', 'trainable': True, 'dtype': 'float32', 'units': 64}


In [25]:
y = linear_layer(tf.ones(shape=(64,3)))
y

<tf.Tensor: shape=(64, 64), dtype=float32, numpy=
array([[-0.03712314, -0.02370307,  0.05135906, ...,  0.14805348,
         0.00862139,  0.18013929],
       [-0.03712314, -0.02370307,  0.05135906, ...,  0.14805348,
         0.00862139,  0.18013929],
       [-0.03712314, -0.02370307,  0.05135906, ...,  0.14805348,
         0.00862139,  0.18013929],
       ...,
       [-0.03712314, -0.02370307,  0.05135906, ...,  0.14805348,
         0.00862139,  0.18013929],
       [-0.03712314, -0.02370307,  0.05135906, ...,  0.14805348,
         0.00862139,  0.18013929],
       [-0.03712314, -0.02370307,  0.05135906, ...,  0.14805348,
         0.00862139,  0.18013929]], dtype=float32)>

In [26]:
# 从配置中构建网络，（已知网络结构，不知超参的情况）
new_layer = Linear.from_config(config)
new_layer

如果在反序列化中(从配置中构建网络)需要更大的灵活性，可以重写from_config方法。

In [27]:
def from_config(cls, config):
    return cls(**config)

### 3.2 配置训练时特有参数
有一些网络层， 如BatchNormalization层和Dropout层，在训练和推理中具有不同的行为，对于此类层，则需要在方法中使用train等参数进行控制。

In [28]:
class MyDropout(layers.Layer):
    def __init__(self, rate, **kwargs):
        super(MyDropout, self).__init__(**kwargs)
        self.rate = rate
    
    def call(self, inputs, training=None):
        return tf.cond(training, 
                    lambda: tf.nn.dropout(inputs, rate=self.rate),
                    lambda: inputs)

## 4 构建自己的模型
通常，我们使用Layer类来定义内部计算块，并使用Model类来定义外部模型 - 即要训练的对象。

Model类与Layer的区别：
- 它对外开放内置的训练，评估和预测函数（model.fit(),model.evaluate(),model.predict()）。 
- 它通过model.layers属性开放其内部网络层列表。 
- 它对外开放保存和序列化API。

### 4.1 自定义模型
下面通过构建一个变分自编码器（VAE），来介绍如何构建自己的网络， 并使用内置的函数进行训练。

In [29]:
# 采样网络
from itertools import zip_longest
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5*z_log_var) * epsilon
    
# 编码器
class Encoder(layers.Layer):
    def __init__(self, latent_dim=32, intermediate_dim=64, name='encoder', **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation='relu')
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        h1 = self.dense_proj(inputs)
        # 获取z_mean或z_log_var
        z_mean = self.dense_mean(h1)
        z_log_var = self.dense_log_var(h1)
        # 进行采样
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z
    
class Decoder(layers.Layer):
    def __init__(self, original_dim, intermediate_dim=64, name='decoder', **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation='relu')
        self.dense_output = layers.Dense(original_dim, activation='sigmoid')
    def call(self, inputs):
        # 两层全连接网络
        h1 = self.dense_proj(inputs)
        return self.dense_output(h1)

class VAE(tf.keras.Model):
    def __init__(self, original_dim, latent_dim=32, intermediate_dim=64, name='encoder', **kwargs):
        super(VAE, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim=original_dim, intermediate_dim=intermediate_dim)
    
    def call(self, inputs):
        # 编码
        z_mean, z_log_var, z = self.encoder(inputs)
        # 解码
        reconstructed = self.decoder(z)
        # 获取损失函数
        kl_loss = -0.5 * tf.reduce_sum(
            z_log_var-tf.square(z_mean)-tf.exp(z_log_var)+1
        )
        self.add_loss(kl_loss)
        return reconstructed

In [30]:

(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
vae = VAE(784, latent_dim=32, intermediate_dim=64)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=3, batch_size=64)

Epoch 1/3
938/938 [==============================] - 3s 2ms/step - loss: 0.7666
Epoch 2/3
938/938 [==============================] - 2s 2ms/step - loss: 0.0688
Epoch 3/3
938/938 [==============================] - 2s 2ms/step - loss: 0.0678


In [31]:
train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)
original_dim= 784
vae = VAE(original_dim, 64, 32)

# 优化器
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
# 损失函数
mse_loss_fn = tf.keras.losses.MeanSquaredError()
# 评价指标
loss_metric = tf.keras.metrics.Mean()

# 训练循环
for epoch in range(3):
    print('Start of epoch %d' % (epoch,))
    for step, x_batch_train in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # 前向传播
            reconstructed = vae(x_batch_train)
            # 计算loss
            loss = mse_loss_fn(x_batch_train, reconstructed)
            loss += sum(vae.losses) # Add KLD regularization loss
        
        # 计算梯度
        grads = tape.gradient(loss, vae.trainable_variables)
        # 反向传播
        optimizer.apply_gradients(zip(grads, vae.trainable_variables))
        # 统计指标
        loss_metric(loss)
        # 输出
        if step % 100 == 0:
            print('step %s: mean loss = %s' % (step, loss_metric.result()))

Start of epoch 0
step 0: mean loss = tf.Tensor(225.1166, shape=(), dtype=float32)
step 100: mean loss = tf.Tensor(6.628161, shape=(), dtype=float32)
step 200: mean loss = tf.Tensor(3.3782306, shape=(), dtype=float32)
step 300: mean loss = tf.Tensor(2.2841198, shape=(), dtype=float32)
step 400: mean loss = tf.Tensor(1.7342669, shape=(), dtype=float32)
step 500: mean loss = tf.Tensor(1.4038577, shape=(), dtype=float32)
step 600: mean loss = tf.Tensor(1.1826853, shape=(), dtype=float32)
step 700: mean loss = tf.Tensor(1.0242989, shape=(), dtype=float32)
step 800: mean loss = tf.Tensor(0.9054826, shape=(), dtype=float32)
step 900: mean loss = tf.Tensor(0.8128643, shape=(), dtype=float32)
Start of epoch 1
step 0: mean loss = tf.Tensor(0.782809, shape=(), dtype=float32)
step 100: mean loss = tf.Tensor(0.7141648, shape=(), dtype=float32)
step 200: mean loss = tf.Tensor(0.65757877, shape=(), dtype=float32)
step 300: mean loss = tf.Tensor(0.61011475, shape=(), dtype=float32)
step 400: mean loss